In [21]:
import hashlib
import time
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from collections import defaultdict
import numpy as np
from scipy.optimize import linprog

# Blockchain structure
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        tx_str = str(self.transactions) + str(self.timestamp) + str(self.previous_hash)
        return hashlib.sha256(tx_str.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.transaction_log = set()

    def create_genesis_block(self):
        return Block(0, [], time.time(), "0")

    def add_block(self, transactions):
        filtered = []
        for tx in transactions:
            tx_id = f"{tx['from']}->{tx['to']}@{tx['energy_kWh']}"
            if tx_id not in self.transaction_log and tx['energy_kWh'] > 0:
                self.transaction_log.add(tx_id)
                filtered.append(tx)
        if not filtered:
            return None  # Return None if no valid transactions
        latest = self.chain[-1]
        new_block = Block(len(self.chain), filtered, time.time(), latest.hash)
        self.chain.append(new_block)
        return new_block  # Return the new block

class SimpleBlockchain(Blockchain):
    def add_block(self, transactions):
        if transactions:
            latest = self.chain[-1]
            new_block = Block(len(self.chain), transactions, time.time(), latest.hash)
            self.chain.append(new_block)
            return new_block
        return None

class HashIndexedBlockchain(Blockchain):
    def __init__(self):
        super().__init__()
        self.index_map = {}

    def add_block(self, transactions):
        filtered = []
        for tx in transactions:
            tx_id = f"{tx['from']}->{tx['to']}@{tx['energy_kWh']}"
            if tx_id not in self.transaction_log and tx['energy_kWh'] > 0:
                self.transaction_log.add(tx_id)
                self.index_map[tx_id] = len(self.chain)
                filtered.append(tx)
        if not filtered:
            return None
        latest = self.chain[-1]
        new_block = Block(len(self.chain), filtered, time.time(), latest.hash)
        self.chain.append(new_block)
        return new_block

class VerifiedBlockchain(Blockchain):
    def add_block(self, transactions):
        verified = []
        for tx in transactions:
            tx_id = f"{tx['from']}->{tx['to']}@{tx['energy_kWh']}"
            if (tx_id not in self.transaction_log and
                tx['energy_kWh'] > 0 and
                tx['from'] != tx['to']):
                self.transaction_log.add(tx_id)
                verified.append(tx)
        if not verified:
            return None
        latest = self.chain[-1]
        new_block = Block(len(self.chain), verified, time.time(), latest.hash)
        self.chain.append(new_block)
        return new_block

# Participants
class Participant:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance

class Prosumer(Participant):
    def __init__(self, name, energy_available, price_per_unit):
        super().__init__(name)
        self.energy_available = energy_available
        self.price_per_unit = price_per_unit

class Consumer(Participant):
    def __init__(self, name, energy_required, bid_price):
        super().__init__(name)
        self.energy_required = energy_required
        self.bid_price = bid_price

# Pricing strategies - MODIFIED TO INCREASE P2P TRADING
def time_strategy(hour):
    if 10 <= hour <= 14:
        return 5.5  # Reduced from 6.5
    elif 7 <= hour <= 9 or 15 <= hour <= 17:
        return 4.5  # Reduced from 5.5
    else:
        return 3.5  # Reduced from 4.5

def inventory_strategy(available):
    if available >= 3.0:
        return 4.0  # Reduced from 5.0
    elif available >= 1.5:
        return 4.5  # Reduced from 5.5
    else:
        return 5.0  # Reduced from 6.0

def solar_generation(hour):
    if 6 <= hour <= 18:
        irradiance = max(0, math.sin(math.pi * (hour - 6) / 12)) * 1.2
        panel_area = 12.63  # Increased from 6.0
        efficiency = 0.165
        return round(irradiance * panel_area * efficiency, 2)
    else:
        return 0

def simulate_market(strategy_type, use_blockchain=True, shared_gen_load=None, blockchain_class=Blockchain):
    profile_data = []
    p2p_matrix = pd.DataFrame(0.0, index=[f"Home_{i}" for i in range(1, 11)], columns=[f"Home_{i}" for i in range(1, 11)])
    consumer_payments = defaultdict(float)
    prosumer_earnings = defaultdict(float)
    market_chain = blockchain_class() if use_blockchain else []

    def match_market(prosumers, consumers):
        adjusted_prosumers = []
        adjusted_consumers = []
        transactions = []

        home_map = {}
        for p in prosumers:
            home_name = p.name.replace("_PV", "")
            home_map[home_name] = {'prosumer': p, 'consumer': None}
        for c in consumers:
            home_map[c.name]['consumer'] = c

        for home_name, data in home_map.items():
            p = data['prosumer']
            c = data['consumer']

            if p and c:
                self.consumption = min(p.energy_available, c.energy_required)
                if self_consumption > 0:
                    c.energy_required -= self_consumption
                    p.energy_available -= self_consumption

            if p and p.energy_available > 0:
                adjusted_prosumers.append(p)
            if c and c.energy_required > 0:
                adjusted_consumers.append(c)

        if adjusted_prosumers and adjusted_consumers:
            sellers = [(i, p.name, p.energy_available, p.price_per_unit)
                       for i, p in enumerate(adjusted_prosumers) if p.energy_available > 0]
            buyers = [(j, c.name, c.energy_required, c.bid_price)
                      for j, c in enumerate(adjusted_consumers) if c.energy_required > 0]

            if sellers and buyers:
                num_sellers = len(sellers)
                num_buyers = len(buyers)
                cost_matrix = np.zeros((num_sellers, num_buyers))

                for i, (_, _, _, seller_price) in enumerate(sellers):
                    for j, (_, _, _, buyer_price) in enumerate(buyers):
                        if buyer_price >= seller_price:
                            cost_matrix[i, j] = -(buyer_price - seller_price)

                c_vec = cost_matrix.flatten()
                A_ub_supply = np.zeros((num_sellers, num_sellers * num_buyers))
                for i in range(num_sellers):
                    A_ub_supply[i, i*num_buyers:(i+1)*num_buyers] = 1
                b_ub_supply = [seller[2] for seller in sellers]

                A_ub_demand = np.zeros((num_buyers, num_sellers * num_buyers))
                for j in range(num_buyers):
                    A_ub_demand[j, j::num_buyers] = 1
                b_ub_demand = [buyer[2] for buyer in buyers]

                A_ub = np.vstack([A_ub_supply, A_ub_demand])
                b_ub = np.hstack([b_ub_supply, b_ub_demand])

                res = linprog(c_vec, A_ub=A_ub, b_ub=b_ub, bounds=(0, None), method='highs')

                if res.success:
                    solution = res.x.reshape((num_sellers, num_buyers))
                    for i in range(num_sellers):
                        for j in range(num_buyers):
                            if solution[i, j] > 0.001:
                                seller = sellers[i]
                                buyer = buyers[j]
                                amount = solution[i, j] * seller[3]
                                transactions.append({
                                    'from': seller[1], 'to': buyer[1],
                                    'energy_kWh': solution[i, j],
                                    'price_per_kWh': seller[3],
                                    'total_amount': amount
                                })
                                consumer_payments[buyer[1]] += amount
                                prosumer_earnings[seller[1]] += amount

        GRID_IMPORT_PRICE = 12
        GRID_EXPORT_PRICE = 2

        total_supply = sum(p.energy_available for p in adjusted_prosumers)
        total_demand = sum(c.energy_required for c in adjusted_consumers)

        unmet_demand = max(0, total_demand - total_supply)
        if unmet_demand > 0:
            grid_import_amount = unmet_demand * GRID_IMPORT_PRICE
            transactions.append({
                'from': 'Grid', 'to': 'System',
                'energy_kWh': unmet_demand,
                'price_per_kWh': GRID_IMPORT_PRICE,
                'total_amount': grid_import_amount
            })
            total_load = sum(c.energy_required for c in adjusted_consumers)
            if total_load > 0:
                for c in adjusted_consumers:
                    if c.energy_required > 0:
                        consumer_payments[c.name] += (c.energy_required / total_load) * grid_import_amount

        excess_supply = max(0, total_supply - total_demand)
        if excess_supply > 0:
            grid_export_amount = excess_supply * GRID_EXPORT_PRICE
            transactions.append({
                'from': 'System', 'to': 'Grid',
                'energy_kWh': excess_supply,
                'price_per_kWh': GRID_EXPORT_PRICE,
                'total_amount': grid_export_amount
            })
            total_gen = sum(p.energy_available for p in adjusted_prosumers)
            if total_gen > 0:
                for p in adjusted_prosumers:
                    if p.energy_available > 0:
                        prosumer_earnings[p.name] += (p.energy_available / total_gen) * grid_export_amount

        return transactions


    # --- MODIFICATION START ---
    # Define fixed total generation and load values
    # We will use these fixed values for every hour of every simulation
    FIXED_TOTAL_GENERATION = 50.0  # Example fixed value for total generation
    FIXED_TOTAL_LOAD = 45.0          # Example fixed value for total load

    for hour in range(24):
        prosumers, consumers = [], []
        
        # Distribute fixed generation and load across homes
        for i in range(1, 11):
            # Non-uniform distribution factors still apply to the fixed total
            load = FIXED_TOTAL_LOAD / 10 + random.uniform(-1, 1)
            gen = FIXED_TOTAL_GENERATION / 10 + random.uniform(-1, 1)

            # Ensure non-negative values
            load = max(0, load)
            gen = max(0, gen)

            name = f"Home_{i}"

            strategy = time_strategy(hour) if strategy_type == 'time' else inventory_strategy(gen - load)
            strategy += random.uniform(-0.3, 0.3)
            strategy = max(4.0, min(6.5, strategy))

            prosumers.append(Prosumer(f"{name}_PV", gen, strategy))
            bid_price = round(random.uniform(8.0, 10.0), 2)
            consumers.append(Consumer(name, load, bid_price))

        total_gen = sum([p.energy_available for p in prosumers])
        total_load = sum([c.energy_required for c in consumers])
        
        profile_data.append({"Hour": f"{hour}:00", "Total_Gen_kWh": total_gen, "Total_Load_kWh": total_load})

        transactions = match_market(prosumers, consumers)
        if transactions:
            if use_blockchain:
                market_chain.add_block(transactions)
            else:
                market_chain.append(transactions)
    # --- MODIFICATION END ---

    df = pd.DataFrame(profile_data)
    grid_import = defaultdict(float)
    grid_export = defaultdict(float)
    p2p_trades = defaultdict(float)

    if use_blockchain:
        all_blocks = market_chain.chain[1:]
        for block in all_blocks:
            hour_str = f"{block.index-1}:00"
            for tx in block.transactions:
                if tx['from'] == 'Grid':
                    grid_import[hour_str] += tx['energy_kWh']
                if tx['to'] == 'Grid':
                    grid_export[hour_str] += tx['energy_kWh']
                if "Home" in tx['from'] and "Home" in tx['to']:
                    p2p_trades[hour_str] += tx['energy_kWh']
    else:
        for hour, txs in enumerate(market_chain):
            hour_str = f"{hour}:00"
            for tx in txs:
                if tx['from'] == 'Grid':
                    grid_import[hour_str] += tx['energy_kWh']
                if tx['to'] == 'Grid':
                    grid_export[hour_str] += tx['energy_kWh']
                if "Home" in tx['from'] and "Home" in tx['to']:
                    p2p_trades[hour_str] += tx['energy_kWh']

    df["Grid_Import_kWh"] = [grid_import.get(h, 0) for h in df["Hour"]]
    df["Grid_Export_kWh"] = [grid_export.get(h, 0) for h in df["Hour"]]
    df["P2P_Trades_kWh"] = [p2p_trades.get(h, 0) for h in df["Hour"]]
    df["Net_Balance"] = (df["Total_Gen_kWh"] + df["Grid_Import_kWh"] - df["Total_Load_kWh"] - df["Grid_Export_kWh"]).round(6)

    p2p_matrix = pd.DataFrame(0.0, index=[f"Home_{i}" for i in range(1, 11)], columns=[f"Home_{i}" for i in range(1, 11)])
    for hour_str in df["Hour"]:
        hour = int(hour_str.split(":")[0])
        txs = []
        if use_blockchain:
            if hour + 1 < len(market_chain.chain):
                txs = market_chain.chain[hour + 1].transactions
        else:
            if hour < len(market_chain):
                txs = market_chain[hour]
        
        for tx in txs:
            if "Home" in tx['from'] and "Home" in tx['to']:
                sender = tx['from'].replace("_PV", "")
                receiver = tx['to']
                if sender in p2p_matrix.index and receiver in p2p_matrix.columns:
                    p2p_matrix.loc[sender, receiver] += tx['energy_kWh']

    return df, market_chain, consumer_payments, prosumer_earnings, p2p_matrix

# New function to calculate average values
def calculate_averages(df):
    return {
        'Average Hourly Generation': df['Total_Gen_kWh'].mean(),
        'Average Hourly Load': df['Total_Load_kWh'].mean()
    }

# Shared profile for fair comparison
shared_gen_load = [round(random.uniform(8.1, 12.1), 2) for _ in range(24)]
comparison_data = []

# Run for all configurations
blockchain_variants = [Blockchain, SimpleBlockchain, HashIndexedBlockchain, VerifiedBlockchain]
variant_names = ["Default", "Simple", "HashIndexed", "Verified"]

for variant_class, variant_name in zip(blockchain_variants, variant_names):
    for strategy in ['time', 'inventory']:
        for blockchain_flag in [True, False]:
            label = f"{strategy.title()} + {variant_name} {'Blockchain' if blockchain_flag else 'No Blockchain'}"
            print(f"=== {label} ===")

            try:
                df_blk, market_chain, consumer_payments, prosumer_earnings, p2p_matrix = simulate_market(
                    strategy, use_blockchain=blockchain_flag, shared_gen_load=shared_gen_load, blockchain_class=variant_class
                )

                # Calculate financial metrics
                total_consumer_payments = sum(consumer_payments.values())
                total_prosumer_earnings = sum(prosumer_earnings.values())
                net_system_cost = total_consumer_payments - total_prosumer_earnings
                
                # Calculate average generation and load
                averages = calculate_averages(df_blk)

                comparison_data.append({
                    'Strategy': label,
                    'Total Generation': round(df_blk['Total_Gen_kWh'].sum(), 2),
                    'Average Hourly Generation': round(averages['Average Hourly Generation'], 2),
                    'Total Load': round(df_blk['Total_Load_kWh'].sum(), 2),
                    'Average Hourly Load': round(averages['Average Hourly Load'], 2),
                    'Grid Import': round(df_blk['Grid_Import_kWh'].sum(), 2),
                    'Grid Export': round(df_blk['Grid_Export_kWh'].sum(), 2),
                    'P2P Trades': round(df_blk['P2P_Trades_kWh'].sum(), 2),
                    'Final Net Balance': round(df_blk['Net_Balance'].sum(), 2),
                    'Total Consumer Payments': round(total_consumer_payments, 2),
                    'Total Prosumer Earnings': round(total_prosumer_earnings, 2),
                    'Net System Cost': round(net_system_cost, 2)
                })

                print(df_blk.sum(numeric_only=True).round(2))

                # P2P matrix heatmap
                plt.figure(figsize=(12, 6))
                sns.heatmap(p2p_matrix, annot=True, fmt=".2f", cmap="YlGnBu")
                plt.title(f"P2P Trade Matrix - {strategy.title()} - {variant_name} {'Blockchain' if blockchain_flag else 'No Blockchain'}")
                plt.xlabel("Receiver")
                plt.ylabel("Sender")
                plt.tight_layout()
                plt.show()

                # Generation, Load, Grid flows
                plt.figure(figsize=(12, 6))
                plt.plot(df_blk["Hour"], df_blk["Total_Gen_kWh"], label="Generation", marker='o')
                plt.plot(df_blk["Hour"], df_blk["Total_Load_kWh"], label="Load", marker='x')
                plt.plot(df_blk["Hour"], df_blk["Grid_Import_kWh"], label="Grid Import", linestyle=':', marker='>')
                plt.plot(df_blk["Hour"], df_blk["Grid_Export_kWh"], label="Grid Export", linestyle=':', marker='<')
                plt.plot(df_blk["Hour"], df_blk["Net_Balance"], label="Final Net Balance", linestyle='--', marker='s')
                plt.title(f"Generation, Load & Power Balancing - {strategy.title()} - {variant_name} {'Blockchain' if blockchain_flag else 'No Blockchain'}")
                plt.xlabel("Hour")
                plt.ylabel("Energy (kWh)")
                plt.grid(True)
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.legend()
                plt.show()

            except Exception as e:
                print(f"Error running {label}: {str(e)}")
                continue

# Print strategy comparison summary
print("\n=== Corrected Energy Market Simulation Results ===")
comparison_df = pd.DataFrame(comparison_data)
comparison_columns = [
    'Strategy', 'Total Generation', 'Average Hourly Generation',
    'Total Load', 'Average Hourly Load', 'Grid Import', 'Grid Export', 'P2P Trades',
    'Final Net Balance', 'Total Consumer Payments',
    'Total Prosumer Earnings', 'Net System Cost'
]
print(comparison_df[comparison_columns].to_string(index=False))

# Blockchain benefit comparison
print("\n=== Blockchain Benefit Analysis ===")
print("| Strategy            | Metric              | With Blockchain | No Blockchain | Difference | Benefit   |")
print("|---------------------|---------------------|------------------|----------------|------------|-----------|")

for i in range(0, len(comparison_data), 2):
    strat = comparison_data[i]['Strategy'].replace(' + Blockchain', '').replace('Default ', '')
    blk = comparison_data[i]
    no_blk = comparison_data[i+1] if i+1 < len(comparison_data) else None

    if no_blk is None:
        continue

    def benefit_text(diff, better_if_positive):
        if metric == "Final Net Balance":
            if abs(blk[metric]) < abs(no_blk[metric]):
                return "Better"
            elif abs(blk[metric]) > abs(no_blk[metric]):
                return "Worse"
            else:
                return "Equal"
        else:
            if (diff > 0 and better_if_positive) or (diff < 0 and not better_if_positive):
                return "Better"
            elif diff == 0:
                return "Equal"
            else:
                return "Worse"

    for metric, better_if_positive in [
        ("P2P Trades", True),
        ("Grid Import", False),
        ("Grid Export", True),
        ("Final Net Balance", False),
        ("Net System Cost", False)]:
        diff = round(blk[metric] - no_blk[metric], 2)
        benefit = benefit_text(diff, better_if_positive)
        print(f"| {strat:<20} | {metric:<20} | {blk[metric]:>16.2f} | {no_blk[metric]:>14.2f} | {diff:>10.2f} | {benefit:<9} |")
        

=== Time + Default Blockchain ===
Error running Time + Default Blockchain: name 'self' is not defined
=== Time + Default No Blockchain ===
Error running Time + Default No Blockchain: name 'self' is not defined
=== Inventory + Default Blockchain ===
Error running Inventory + Default Blockchain: name 'self' is not defined
=== Inventory + Default No Blockchain ===
Error running Inventory + Default No Blockchain: name 'self' is not defined
=== Time + Simple Blockchain ===
Error running Time + Simple Blockchain: name 'self' is not defined
=== Time + Simple No Blockchain ===
Error running Time + Simple No Blockchain: name 'self' is not defined
=== Inventory + Simple Blockchain ===
Error running Inventory + Simple Blockchain: name 'self' is not defined
=== Inventory + Simple No Blockchain ===
Error running Inventory + Simple No Blockchain: name 'self' is not defined
=== Time + HashIndexed Blockchain ===
Error running Time + HashIndexed Blockchain: name 'self' is not defined
=== Time + HashInd

KeyError: "None of [Index(['Strategy', 'Total Generation', 'Average Hourly Generation',\n       'Total Load', 'Average Hourly Load', 'Grid Import', 'Grid Export',\n       'P2P Trades', 'Final Net Balance', 'Total Consumer Payments',\n       'Total Prosumer Earnings', 'Net System Cost'],\n      dtype='object')] are in the [columns]"